In [ ]:
# device = torch.device("cuda" if torch.cuda.is_available () else "cpu")

In [ ]:
import gc
from PIL import Image
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets,transforms
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
plt.rcParams["figure.figsize"]= 15,10
#from layer import KernelConv2d, GaussianKernel, PolynomialKernel
from functools import partial # To invoke Kernel objects with input parameters when creating KernelConv2d object (e.g. partial(GaussianKernel, 0.05) for Gaussian OR partial(PolynomialKernel,2,3) for Polynomial)
%matplotlib inline
def mkdirs(path):
    if not os.path.exists(path):
        os.makedirs(path)
        

In [ ]:
seed = 17
torch.manual_seed(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [ ]:
class Kernel(nn.Module):
    def __init__(self,in_channel,out_channel,kernelsize,kernel_fn, c=1.0,degree=5,gamma = 0.5,rhok=0.02):
        super(Kernel, self).__init__()
        self.conv1 = nn.Conv2d(in_channel,out_channel,kernelsize)
        
        
        if kernel_fn == 0:            
            self.c = torch.nn.parameter.Parameter(torch.tensor(c), requires_grad=False)
            self.degree = torch.nn.parameter.Parameter(torch.tensor(degree), requires_grad=False)
        
        if kernel_fn == 1:
            self.gamma = torch.nn.parameter.Parameter(torch.tensor(gamma), requires_grad=False)
        
        if kernel_fn >= 3:
            self.rho = torch.nn.parameter.Parameter(torch.tensor(rhok, requires_grad=True))   
            self.c = torch.nn.parameter.Parameter(torch.tensor(c), requires_grad=False)
            self.degree = torch.nn.parameter.Parameter(torch.tensor(degree), requires_grad=False)
            self.gamma = torch.nn.parameter.Parameter(torch.tensor(gamma), requires_grad=False)
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernelsize = kernelsize
        self.kernel_fn= kernel_fn
 
    
    def __compute_shape(self, x):
        h = (x.shape[2] - self.conv1.kernel_size[0] + 2 * self.conv1.padding[0]) // self.conv1.stride[0] + 1
        w = (x.shape[3] - self.conv1.kernel_size[1] + 2 * self.conv1.padding[1]) // self.conv1.stride[1] + 1
        return h, w
        
    def convolution(self,x):
        return self.conv1(x)
        
    def sigmoidkerv(self,x):
        return torch.tanh(self.convolution(x))
    
    def polynomial(self,x):
#         print(torch.max(self.convolution(x)))
#         print(torch.min(self.convolution(x)))
#         print(torch.max((self.convolution(x) + self.c) ** self.degree))
#         print(torch.min((self.convolution(x) + self.c) ** self.degree))
        return (self.convolution(x) + self.c) ** self.degree
    
    def gaussian(self,x):        
        x_unf = F.unfold(x, self.conv1.kernel_size, self.conv1.dilation, self.conv1.padding, self.conv1.stride).transpose(1, 2)
        h, w = self.__compute_shape(x)
        l2 = x_unf.unsqueeze(3) - self.conv1.weight.view(1, 1, -1, self.conv1.weight.size(0))
        l2 = torch.sum(l2 ** 2, 2)
        out =  torch.exp(-self.gamma * l2)
        if self.conv1.bias is not None:
            out = out + self.conv1.bias
        return out.view(x.shape[0], self.conv1.out_channels, w, h)

    def polyconv(self,x):
        conv = self.convolution(x)
        return torch.sigmoid(self.rho)*((conv + self.c) ** self.degree) + (1-torch.sigmoid(self.rho))*conv
    
    def polysigm(self,x):
        conv = self.convolution(x)
        return torch.sigmoid(self.rho)*((conv + self.c) ** self.degree) + (1-torch.sigmoid(self.rho))*torch.tanh(conv)
        
    def kernel_fn_a(self, x):
        #Polynomial
        return F.relu(self.polynomial(x))
    
    def kernel_fn_b(self,x):
        #Gaussian
        return F.relu(self.gaussian(x))
    
    def kernel_fn_c(self, x):
        #Sigmoid
        return F.relu(self.sigmoidkerv(x))
    
    def kernel_fn_d(self, x):
        #Polynomial + Convolution
        return F.relu(self.polyconv(x))
    
    def kernel_fn_e(self, x):
        #Sigmoid + Convolution
        conv = self.convolution(x)
        return F.relu(torch.sigmoid(self.rho)*torch.tanh(conv) + (1- torch.sigmoid(self.rho)) * conv)
        
    def kernel_fn_f(self, x):
        #Gaussian + Convolution
        return F.relu(torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * self.convolution(x))
        
    def kernel_fn_g(self, x):
        #Gaussian + Polynomial
        return F.relu(torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * self.polynomial(x))
      
        
    def kernel_fn_h(self, x):
        #Gaussian + Sigmoid
        return F.relu(torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * self.sigmoidkerv(x))
        
    def kernel_fn_i(self, x):
        #Polynomial + Sigmoid##
        return F.relu(self.polysigm(x))
        
    
    def forward(self, x):

        json = {0: self.kernel_fn_a ,1: self.kernel_fn_b ,2: self.kernel_fn_c ,
                3: self.kernel_fn_d ,4: self.kernel_fn_e , 5: self.kernel_fn_f,
                6: self.kernel_fn_g ,7: self.kernel_fn_h, 8: self.kernel_fn_i }
    
        function =  self.kernel_fn
        return json[function](x)
    
 


In [ ]:
class Kernel_norelu(nn.Module):
    def __init__(self,in_channel,out_channel,kernelsize,kernel_fn, c=1.0,degree=5,gamma = 0.5,rhok=0.02):
        super(Kernel_norelu, self).__init__()
        self.conv1 = nn.Conv2d(in_channel,out_channel,kernelsize)
        
        
        if kernel_fn == 0:            
            self.c = torch.nn.parameter.Parameter(torch.tensor(c), requires_grad=False)
            self.degree = torch.nn.parameter.Parameter(torch.tensor(degree), requires_grad=False)
        
        if kernel_fn == 1:
            self.gamma = torch.nn.parameter.Parameter(torch.tensor(gamma), requires_grad=False)
        
        if kernel_fn >= 3:
            self.rho = torch.nn.parameter.Parameter(torch.tensor(rhok, requires_grad=True))   
            self.c = torch.nn.parameter.Parameter(torch.tensor(c), requires_grad=False)
            self.degree = torch.nn.parameter.Parameter(torch.tensor(degree), requires_grad=False)
            self.gamma = torch.nn.parameter.Parameter(torch.tensor(gamma), requires_grad=False)
        
        self.in_channel = in_channel
        self.out_channel = out_channel
        self.kernelsize = kernelsize
        self.kernel_fn= kernel_fn
 
    
    def __compute_shape(self, x):
        h = (x.shape[2] - self.conv1.kernel_size[0] + 2 * self.conv1.padding[0]) // self.conv1.stride[0] + 1
        w = (x.shape[3] - self.conv1.kernel_size[1] + 2 * self.conv1.padding[1]) // self.conv1.stride[1] + 1
        return h, w
        
    def convolution(self,x):
        return (self.conv1(x))
    def sigmoidkerv(self,x):
        return torch.tanh(self.convolution(x))
    
    def polynomial(self,x):
#         print(torch.max(self.convolution(x)))
#         print(torch.min(self.convolution(x)))
#         print(torch.max((self.convolution(x) + self.c) ** self.degree))
#         print(torch.min((self.convolution(x) + self.c) ** self.degree))
        return (self.convolution(x) + self.c) ** self.degree
    
    def gaussian(self,x):        
        x_unf = F.unfold(x, self.conv1.kernel_size, self.conv1.dilation, self.conv1.padding, self.conv1.stride).transpose(1, 2)
        h, w = self.__compute_shape(x)
        l2 = x_unf.unsqueeze(3) - self.conv1.weight.view(1, 1, -1, self.conv1.weight.size(0))
        l2 = torch.sum(l2 ** 2, 2)
        out =  torch.exp(-self.gamma * l2)
        if self.conv1.bias is not None:
            out = out + self.conv1.bias
        return out.view(x.shape[0], self.conv1.out_channels, w, h)

    def polyconv(self,x):
        conv = self.convolution(x)
        return torch.sigmoid(self.rho)*((conv + self.c) ** self.degree) + (1-torch.sigmoid(self.rho))*(conv)
    
    def polysigm(self,x):
        conv = self.convolution(x)
        return torch.sigmoid(self.rho)*((conv + self.c) ** self.degree) + (1-torch.sigmoid(self.rho))*torch.tanh(conv)
        
    def kernel_fn_a(self, x):
        #Polynomial
        return (self.polynomial(x))
    
    def kernel_fn_b(self,x):
        #Gaussian
        return (self.gaussian(x))
    
    def kernel_fn_c(self, x):
        #Sigmoid
        return (self.sigmoidkerv(x))
    
    def kernel_fn_d(self, x):
        #Polynomial + Convolution
        return (self.polyconv(x))
    
    def kernel_fn_e(self, x):
        #Sigmoid + Convolution
        conv = self.convolution(x)
        return (torch.sigmoid(self.rho)*torch.tanh(conv) + (1- torch.sigmoid(self.rho)) * (conv))
        
    def kernel_fn_f(self, x):
        #Gaussian + Convolution
        return (torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * (self.convolution(x)))
        
    def kernel_fn_g(self, x):
        #Gaussian + Polynomial
        return (torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * self.polynomial(x))
      
        
    def kernel_fn_h(self, x):
        #Gaussian + Sigmoid
        return (torch.sigmoid(self.rho)*self.gaussian(x) + (1- torch.sigmoid(self.rho)) * self.sigmoidkerv(x))
        
    def kernel_fn_i(self, x):
        #Polynomial + Sigmoid##
        return (self.polysigm(x))
        
    
    def forward(self, x):

        json = {0: self.kernel_fn_a ,1: self.kernel_fn_b ,2: self.kernel_fn_c ,
                3: self.kernel_fn_d ,4: self.kernel_fn_e , 5: self.kernel_fn_f,
                6: self.kernel_fn_g ,7: self.kernel_fn_h, 8: self.kernel_fn_i }
    
        function =  self.kernel_fn
        return json[function](x)
    
 


In [ ]:
# class LeNet5MNIST_Conv(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Conv,self).__init__()
#         self.conv1=nn.Conv2d(1,6,5 ) 
#         self.conv2=nn.Conv2d(6,16,5 )  
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)
    
# class LeNet5MNIST_Polynomial(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Polynomial,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
# class LeNet5MNIST_Gaussian(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Gaussian,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=1) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=1) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
    
# class LeNet5MNIST_Sigmoid(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Sigmoid,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
# class LeNet5MNIST_ConvPoly(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_ConvPoly,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)
    
# class LeNet5MNIST_ConvGauss(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_ConvGauss,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=5) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    

# class LeNet5MNIST_ConvSigmoid(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_ConvSigmoid,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=4) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=4) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1) 
    
# class LeNet5MNIST_SigmoidGauss(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_SigmoidGauss,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=7) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=7) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
# class LeNet5MNIST_SigmoidPoly(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_SigmoidPoly,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=8) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=8) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)    
  

# class LeNet5MNIST_GaussPoly(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_GaussPoly,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=6) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=6) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
    
# class LeNet5MNIST_Poly_Conv(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Poly_Conv,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
#         self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)  
    
 
# class LeNet5MNIST_Conv_Poly(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Conv_Poly,self).__init__()
#         self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) 
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=0)
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1) 
    
# class LeNet5MNIST_ConvPoly_Conv(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_ConvPoly_Conv,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)
    
    
# class LeNet5MNIST_Conv_ConvPoly(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Conv_ConvPoly,self).__init__()
#         self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)

    
# class LeNet5MNIST_Conv_Sigmoid(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Conv_Sigmoid,self).__init__()
#         self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=Kernel(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)
    
# class LeNet5MNIST_Sigmoid_Conv(nn.Module):
#     def __init__(self):
#         super(LeNet5MNIST_Sigmoid_Conv,self).__init__()
#         self.conv1=Kernel(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
#         self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
#         self.fc1=nn.Linear(16*4*4,120)
#         self.fc2=nn.Linear(120,84)
#         self.fc3 = nn.Linear(84,10)
#     def forward(self,x):
#         x=F.relu(F.max_pool2d(self.conv1(x),2,stride=2))
#         x=F.relu(F.max_pool2d(self.conv2(x),2,stride=2))
#         x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
#         x=F.relu(self.fc1(x))
#         x=F.relu(self.fc2(x))
#         x=F.relu(self.fc3(x))
#         return F.log_softmax(x,dim=1)
    
    

In [ ]:
class LeNet5MNIST_Conv(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Conv,self).__init__()
        self.conv1=nn.Conv2d(1,6,5 ) 
        self.conv2=nn.Conv2d(6,16,5 )  
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)
    
class LeNet5MNIST_Polynomial(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Polynomial,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
class LeNet5MNIST_Gaussian(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Gaussian,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=1) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=1) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
    
class LeNet5MNIST_Sigmoid(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Sigmoid,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
class LeNet5MNIST_ConvPoly(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_ConvPoly,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)
    
class LeNet5MNIST_ConvGauss(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_ConvGauss,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=5) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    

class LeNet5MNIST_ConvSigmoid(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_ConvSigmoid,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=4) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=4) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1) 
    
class LeNet5MNIST_SigmoidGauss(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_SigmoidGauss,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=7) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=7) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
class LeNet5MNIST_SigmoidPoly(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_SigmoidPoly,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=8) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=8) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)    
  

class LeNet5MNIST_GaussPoly(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_GaussPoly,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=6) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=6) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
    
class LeNet5MNIST_Poly_Conv(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Poly_Conv,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=0) 
        self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)  
    
 
class LeNet5MNIST_Conv_Poly(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Conv_Poly,self).__init__()
        self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) 
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=0)
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1) 
    
class LeNet5MNIST_ConvPoly_Conv(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_ConvPoly_Conv,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)
    
    
class LeNet5MNIST_Conv_ConvPoly(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Conv_ConvPoly,self).__init__()
        self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=3,kernel_fn=3) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)

    
class LeNet5MNIST_Conv_Sigmoid(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Conv_Sigmoid,self).__init__()
        self.conv1=nn.Conv2d(in_channel=1,out_channel= 6,kernelsize= 5) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=Kernel_norelu(in_channel=6,out_channel= 16,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)
    
class LeNet5MNIST_Sigmoid_Conv(nn.Module):
    def __init__(self):
        super(LeNet5MNIST_Sigmoid_Conv,self).__init__()
        self.conv1=Kernel_norelu(in_channel=1,out_channel= 6,kernelsize= 5,c=1.0,degree=5,kernel_fn=2) # self.conv1=KernelConv2d(1,10,5) for default/Ploynomial kernel with default parameters
        self.conv2=nn.Conv2d(in_channel=6,out_channel= 16,kernelsize= 5) 
        self.fc1=nn.Linear(16*4*4,120)
        self.fc2=nn.Linear(120,84)
        self.fc3 = nn.Linear(84,10)
    def forward(self,x):
        x=(F.avg_pool2d(self.conv1(x),2,stride=2))
        x=(F.avg_pool2d(self.conv2(x),2,stride=2))
        x=x.reshape(-1,16*4*4)#x.view(-1,320)#320
        x=(self.fc1(x))
        x=(self.fc2(x))
        x=(self.fc3(x))
        return F.log_softmax(x,dim=1)
    
    

In [ ]:

def test():
    net = LeNet5MNIST_ConvPoly()
    x = torch.randn(1,1,28,28)
    y = net(x)
#     make_dot(y)
    print(y.size())
test()

torch.Size([1, 10])


In [ ]:
from torchvision import models
model = LeNet5MNIST_ConvPoly()

In [ ]:
# print(model.conv1.kernel_fn._parameters)

In [ ]:
with torch.no_grad():
    train_loader=torch.utils.data.DataLoader(
    datasets.MNIST("data",train=True,download=True,transform=transforms.Compose([
                transforms.ToTensor(),torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])),batch_size=50,shuffle=True)
    test_loader=torch.utils.data.DataLoader(
    datasets.MNIST("data",train=False,download=True,transform=transforms.Compose([
                transforms.ToTensor(), torchvision.transforms.Normalize((0.1307,), (0.3081,))
            ])),batch_size=50,shuffle=False)
print(len(train_loader))
print(len(test_loader))

Extracting data/MNIST/raw/train-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/train-labels-idx1-ubyte.gz to data/MNIST/raw



Extracting data/MNIST/raw/t10k-images-idx3-ubyte.gz to data/MNIST/raw


Extracting data/MNIST/raw/t10k-labels-idx1-ubyte.gz to data/MNIST/raw
Processing...
Done!
1200
200


/pytorch/torch/csrc/utils/tensor_numpy.cpp:141: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program.


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available () else "cpu")

In [ ]:
criterion = nn.NLLLoss() 
model.to(device)

LeNet5MNIST_ConvPoly(
  (conv1): Kernel_norelu(
    (conv1): Conv2d(1, 6, kernel_size=(5, 5), stride=(1, 1))
  )
  (conv2): Kernel_norelu(
    (conv1): Conv2d(6, 16, kernel_size=(5, 5), stride=(1, 1))
  )
  (fc1): Linear(in_features=256, out_features=120, bias=True)
  (fc2): Linear(in_features=120, out_features=84, bias=True)
  (fc3): Linear(in_features=84, out_features=10, bias=True)
)

In [ ]:
# pytorch_total_learn_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
# pytorch_total_learn_params

In [ ]:
# pytorch_total_params = sum(p.numel() for p in model.parameters())
# pytorch_total_params

In [ ]:
# model.conv1.conv1._parameters

In [ ]:
# model.parameters()

In [ ]:
# pytorch_total_params = sum(p.numel() for p in model.conv1._parameters)

In [ ]:
! pip install barbar
import time
from barbar import Bar

nb_epochs=20
# torch.manual_seed(42)


def compute_accuray(pred,true):
    pred_idx=pred.argmax(dim=1).detach().cpu().numpy()
    tmp=pred_idx==true.cpu().numpy()
    return (sum(tmp)/len(pred_idx))*100

def plot_loss_epoch(train_loss, test_loss,epochs, model):
    train, =plt.plot(range(1,epochs+1),train_loss, marker='o')
    test, = plt.plot(range(1, epochs+1),test_loss,  marker="o")
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.legend([train, test],["train_loss","test_loss"])
    plt.title("Loss Vs Epoch for: " + model)
    plt.show()
    
def plot_accuracy_epoch(train_accuracy, test_accuracy,epochs, model):
    train, =plt.plot(range(1,epochs+1),train_accuracy, marker="o")
#     print(train_accuracy)
    test, = plt.plot(range(1, epochs+1),test_accuracy,  marker="o")
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy')
    plt.legend([train, test],["train_accuracy","test_accuracy"])
    plt.title("Accuracy Vs Epoch for: " + model)
    plt.show() 

def model_comparison(test_acc, times, epochs):
    for model, acc in test_acc.items():
        plt.plot(times[model],acc,  marker="o")
        plt.xlabel('Time')
        plt.ylabel('Validation_Accuracy')
        plt.title("Validation Accuracy Vs Time")
        plt.legend(test_acc.keys())
        plt.show()
    
    for model, acc in test_acc.items():
        plt.plot(range(1,epochs+1),acc,  marker="o")
        plt.xlabel('Epochs')
        plt.ylabel('Validation_Accuracy')
        plt.title("Validation Accuracy Vs Epoch")
        plt.legend(test_acc.keys())
        plt.show()
    

    


def train(m,out_dir):
    iter_loss=[]
    train_losses=[]
    test_losses=[]
    train_accuracy=[]
    test_accuracy=[]
    times = []
    
    iter_loss_path=os.path.join(out_dir,"iter_loss.csv")
    epoch_loss_path=os.path.join(out_dir,"epoch_loss.csv")
    last_loss=99999
    mkdirs(os.path.join(out_dir,"models"))
    optimizer=optim.SGD(m.parameters(), lr=0.03)
    
    scheduler = torch.optim.lr_scheduler.MultiStepLR(optimizer, milestones=[5,18], gamma=0.1)
    start_time = time.time()
    for epoch in range(nb_epochs):
        train_loss=0.
        train_acc=0.
        m.train(mode=True)
        for data,target in Bar(train_loader):
            data,target=data.to(device),target.to(device)
            optimizer.zero_grad()
            output=m(data)
            loss=criterion(output,target)
            loss_value=loss.item()
            iter_loss.append(loss_value)
            train_loss+=loss_value
            loss.backward()
            torch.nn.utils.clip_grad_norm_(m.parameters(), 1)
            optimizer.step()
            acc=compute_accuray(torch.exp(output),target)
            train_acc+=acc
            
            
        scheduler.step()

        train_losses.append(train_loss/len(train_loader))
        train_accuracy.append(round(train_acc/len(train_loader),2))
        
        test_loss=0.
        test_acc=0.
        m.train(mode=False)
        with torch.no_grad():
            for data,target in test_loader:
                data,target=data.to(device),target.to(device)
                output=m(data)
                loss=criterion(output,target)
                loss_value=loss.item()
                iter_loss.append(loss_value)
                test_loss+=loss_value
                acc=compute_accuray(torch.exp(output),target)
                test_acc+=acc
            
        time_elapsed = np.round(time.time() - start_time,2)
        test_losses.append(test_loss/len(test_loader))
        test_accuracy.append(round(test_acc/len(test_loader),2))
        times.append(time_elapsed)
        
        print("Epoch {}: train loss is {}, train accuracy is {}; test loss is {}, test accuracy is {}, lr is: {}" .
              format(epoch,round(train_loss/len(train_loader),2),
                     round(train_acc/len(train_loader),2),
                     round(test_loss/len(test_loader),2),
                     round(test_acc/len(test_loader),2),
                     optimizer.param_groups[0]['lr']))        
        if test_loss/len(test_loader)<last_loss:      
            name = str(out_dir) + '_' + ".pth"
            save_model_path=os.path.join(out_dir,name)
            torch.save(m, save_model_path)
            last_loss=test_loss/len(test_loader)
        
#     df=pd.DataFrame()
#     df["iteration"]=np.arange(0,len(iter_loss))
#     df["loss"]=iter_loss
#     df.to_csv(iter_loss_path,index=False)
    
#     df=pd.DataFrame()
#     df["epoch"]=np.arange(0,nb_epochs)
#     df["train_loss"]=train_losses
#     df["test_loss"]=test_losses
#     df.to_csv(epoch_loss_path,index=False)
    
    
#     plot_accuracy_epoch(train_accuracy, test_accuracy, nb_epochs)
#     plot_loss_epoch(train_losses, test_losses, nb_epochs)
    
    return train_accuracy, test_accuracy, train_losses, test_losses, times
    

In [ ]:
train_accuracy, test_accuracy, train_losses, test_losses, times = train(model, "LeNet5MNIST_ConvPoly")

60000/60000: [===============================>] - ETA 0.4s
Epoch 0: train loss is 0.32, train accuracy is 90.68; test loss is 0.11, test accuracy is 96.93, lr is: 0.03
  750/60000: [>...............................] - ETA 17.2s

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LeNet5MNIST_ConvPoly. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Kernel_norelu. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


60000/60000: [===============================>] - ETA 0.0s
Epoch 1: train loss is 0.1, train accuracy is 97.13; test loss is 0.07, test accuracy is 97.91, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 2: train loss is 0.08, train accuracy is 97.86; test loss is 0.06, test accuracy is 98.27, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 3: train loss is 0.06, train accuracy is 98.19; test loss is 0.09, test accuracy is 97.26, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 4: train loss is 0.06, train accuracy is 98.46; test loss is 0.05, test accuracy is 98.67, lr is: 0.003
60000/60000: [===============================>] - ETA 0.0s
Epoch 5: train loss is 0.04, train accuracy is 99.06; test loss is 0.04, test accuracy is 98.86, lr is: 0.003
60000/60000: [===============================>] - ETA 0.0s
Epoch 6: train loss is 0.03, train accuracy is 99.16; test loss is 0.04, test accuracy is 98.79, lr 

In [ ]:
report = pd.DataFrame({"Epochs":range(1,nb_epochs+1),"Train_Accuracy":train_accuracy, "Test_Accuracy":test_accuracy,"Train_Loss":train_losses,"Test_Loss":test_losses, "Time":times})
report.to_csv("LeNet5MNIST_ConvPoly.csv", index=False)

In [ ]:
train_accuracy, test_accuracy, train_losses, test_losses, times = train(model, "Lenet5CATDOG_Polynomial")

60000/60000: [===============================>] - ETA 0.0s
Epoch 0: train loss is 0.05, train accuracy is 98.75; test loss is 0.06, test accuracy is 98.66, lr is: 0.03
  900/60000: [>...............................] - ETA 16.1s

/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type LeNet5MNIST_ConvPoly. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "
/usr/local/lib/python3.6/dist-packages/torch/serialization.py:402: UserWarning: Couldn't retrieve source code for container of type Kernel_norelu. It won't be checked for correctness upon loading.
  "type " + obj.__name__ + ". It won't be checked "


60000/60000: [===============================>] - ETA 0.0s
Epoch 1: train loss is 0.04, train accuracy is 98.82; test loss is 0.06, test accuracy is 98.57, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 2: train loss is 0.04, train accuracy is 98.87; test loss is 0.04, test accuracy is 98.95, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 3: train loss is 0.04, train accuracy is 98.92; test loss is 0.05, test accuracy is 98.69, lr is: 0.03
60000/60000: [===============================>] - ETA 0.0s
Epoch 4: train loss is 0.04, train accuracy is 99.03; test loss is 0.05, test accuracy is 98.68, lr is: 0.003
60000/60000: [===============================>] - ETA 0.0s
Epoch 5: train loss is 0.02, train accuracy is 99.44; test loss is 0.04, test accuracy is 99.0, lr is: 0.003
60000/60000: [===============================>] - ETA 0.0s
Epoch 6: train loss is 0.02, train accuracy is 99.51; test loss is 0.04, test accuracy is 99.03, lr 

In [ ]:
model.conv2._parameters

OrderedDict([('rho', Parameter containing:
              tensor(0.0449, device='cuda:0', requires_grad=True)),
             ('c', Parameter containing:
              tensor(1., device='cuda:0')),
             ('degree', Parameter containing:
              tensor(3, device='cuda:0')),
             ('gamma', Parameter containing:
              tensor(0.5000, device='cuda:0'))])

In [ ]:
torch.sigmoid(torch.tensor(0.0448))

tensor(0.5112)

In [ ]:
# print(model.conv1.convK1.weight[0:1,0:1])
# print(model.conv1.weight[0:1,0:1])

In [ ]:
# print(model.conv1.kernel_fn._parameters)

In [ ]:
# print(model.conv1._parameters)

In [ ]:

import torch
model = torch.load("/kaggle/input/mnist-models/New folder/Lenet5MNIST_Polynomial(1,3)_.pth", map_location = torch.device("cpu"))
model.eval()

In [ ]:
torch.sigmoid(torch.tensor(1.2526))

In [ ]:
def imshowMNIST(img, label):
  
    img = img.numpy()
    img = img.reshape(28,28)
    print(label)
    plt.axis("off")
    fig = plt.figure
    plt.imshow(img, cmap="gray")
    plt.show()

In [ ]:
layers= {"conv1": model.conv1, "conv2": model.conv2.conv1}

In [ ]:
images= []
labels=[]
for img, label in train_loader:
    images.append(img)
    labels.append(label)
    

In [ ]:
image_seven = images[0][labels[0]==7]
image_one = images[0][labels[0]==1]
image_three = images[0][labels[0]==3]

In [ ]:
image_seven.shape

In [ ]:
print(images[0][3].shape)
# print(labels[0][1])
image = images[0][3]
label = labels[0][3]
imshowMNIST(images[0][3],labels[0][3])
# images[0][1].numpy().transpose(1,2,0)

In [ ]:
# ax, fig = plt.subplots()
imshowMNIST(image_seven[0],labels[0][3])


In [ ]:
imshowMNIST(image_one[3],labels[0][3])

In [ ]:
seven = image_seven[0]
one = image_one[3]

## Visualization

#### Filter Maps

In [ ]:
def plot_filters_single_channel_big(t):
    
    #setting the rows and columns
    nrows = t.shape[0]*t.shape[2]
    ncols = t.shape[1]*t.shape[3]
    
    
    npimg = np.array(t.numpy(), np.float32)
    npimg = npimg.transpose((0, 2, 1, 3))
    npimg = npimg.ravel().reshape(nrows, ncols)
    
    npimg = npimg.T
    
    fig, ax = plt.subplots()    
    imgplot = sns.heatmap(npimg, xticklabels=False, yticklabels=False, cmap='gray', ax=ax, cbar=False)
    
    
def plot_filters_single_channel(t):
    
    #kernels depth * number of kernels
    nplots = t.shape[0]*t.shape[1]
    ncols = 12
    
    nrows = 1 + nplots//ncols
    #convert tensor to numpy image
    npimg = np.array(t.numpy(), np.float32)
    
    count = 0
    fig = plt.figure(figsize=(ncols, nrows))
    
    #looping through all the kernels in each channel
    for i in range(t.shape[0]):
        for j in range(t.shape[1]):
            count += 1
            ax1 = fig.add_subplot(nrows, ncols, count)
            npimg = np.array(t[i, j].numpy(), np.float32)
            npimg = (npimg - np.mean(npimg)) / np.std(npimg)
            npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
            ax1.imshow(npimg)
            ax1.set_title(str(i) + ',' + str(j))
            ax1.axis('off')
            ax1.set_xticklabels([])
            ax1.set_yticklabels([])
   
    plt.tight_layout()
    plt.show()
    
    
    
    
def plot_filters_multi_channel(t):
    
    #get the number of kernals
    num_kernels = t.shape[0]    
    
    #define number of columns for subplots
    num_cols = 12
    #rows = num of kernels
    num_rows = num_kernels
    
    #set the figure size
    fig = plt.figure(figsize=(num_cols,num_rows))
    
    #looping through all the kernels
    for i in range(t.shape[0]):
        ax1 = fig.add_subplot(num_rows,num_cols,i+1)
        
        #for each kernel, we convert the tensor to numpy 
        npimg = np.array(t[i].numpy(), np.float32)
        #standardize the numpy image
        npimg = (npimg - np.mean(npimg)) / np.std(npimg)
        npimg = np.minimum(1, np.maximum(0, (npimg + 0.5)))
        npimg = npimg.transpose((1, 2, 0))
        ax1.imshow(npimg)
        ax1.axis('off')
        ax1.set_title(str(i))
        ax1.set_xticklabels([])
        ax1.set_yticklabels([])
        
    plt.savefig('myimage.png', dpi=100)    
    plt.tight_layout()
    plt.show()
    
    
    
    
def plot_weights(model, layer_name, single_channel = True, collated = False):
  
  #extracting the model features at the particular layer number
    layer = layers[layer_name]

      #checking whether the layer is convolution layer or not 
    if isinstance(layer, nn.Conv2d):
        #getting the weight tensor data
        weight_tensor = layer.weight.data

        if single_channel:
            if collated:
                plot_filters_single_channel_big(weight_tensor)
            else:
                plot_filters_single_channel(weight_tensor)

        else:
            print(weight_tensor.shape)
            if weight_tensor.shape[1] == 3:
                plot_filters_multi_channel(weight_tensor)
            else:
                print("Can only plot weights with three channels with single channel = False")

    else:
        print("Can only visualize layers which are convolutional")
        
#visualize weights for alexnet - first conv layer
# plot_weights(alexnet, 0, single_channel = False)

In [ ]:

import seaborn as sns
plot_weights(model = model, layer_name= "conv1", single_channel = True, collated = False)



In [ ]:
plot_weights(model = model, layer_name= "conv2", single_channel = True, collated = False)


In [ ]:
# plot_weights(model = model, layer_name= "conv1", single_channel = False, collated = False)

In [ ]:
plot_weights(model = model, layer_name= "conv1", single_channel = True, collated = True)

In [ ]:
plot_weights(model = model, layer_name= "conv2", single_channel = True, collated = True)

#### Occlusion Experiments







In [ ]:
#custom function to conduct occlusion experiments

def occlusion(model, image, label, occ_size = 15, occ_stride = 5, occ_pixel = 0.5):
  
    #get the width and height of the image
    image_copy = image.reshape(28,28)
    width, height = image_copy.shape[0], image_copy.shape[1]
    print(width)
    print(height)
  
    #setting the output image width and height
    output_height = int(np.ceil((height-occ_size)/occ_stride))
    print("output_height: ", output_height)
    output_width = int(np.ceil((width-occ_size)/occ_stride))
    print("output_width: ", output_width)
    #create a white image of sizes we defined
    heatmap = torch.zeros((output_height, output_width))
    
    #iterate all the pixels in each column
    for h in range(0, height):
        for w in range(0, width):
            
            h_start = h*occ_stride
            w_start = w*occ_stride
            h_end = min(height, h_start + occ_size)
            w_end = min(width, w_start + occ_size)
            
            if (w_end) >= width or (h_end) >= height:
                continue
            
            input_image = image.clone().detach()
            input_image = input_image.reshape(1,1,28,28)
            
            #replacing all the pixel information in the image with occ_pixel(grey) in the specified location
            input_image[:,:, w_start:w_end, h_start:h_end] = occ_pixel
            
            #run inference on modified image
            
            output = torch.exp(model(input_image))
#             print(output.tolist())
            prob = output.tolist()[0][label]
            print(prob)
            
            #setting the heatmap location to probability value
            heatmap[h, w] = prob 

    return heatmap

In [ ]:
imshowMNIST(images[0][1],labels[0][1])

In [ ]:
heatmap =  occlusion(model, image, label.item())
imgplot = sns.heatmap(heatmap, xticklabels=False, yticklabels=False)
figure = imgplot.get_figure()

> ####  Activation Maps

In [ ]:
 list(model.children())

In [ ]:
activation = {}
def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook



model.conv1.register_forward_hook(get_activation('conv1'))
model.conv2.register_forward_hook(get_activation('conv2'))
model.fc1.register_forward_hook(get_activation('fc1'))
model.fc2.register_forward_hook(get_activation('fc2'))
model.fc3.register_forward_hook(get_activation('fc3'))
x = one.reshape(1,1,28,28)
output = model(x)
print(activation['conv1'].shape)

In [ ]:
# activation

In [ ]:
act = activation['conv1'].squeeze()
print(act.shape)
fig, axarr = plt.subplots(act.size(0), figsize=(10,10))
for idx in range(act.size(0)):
    axarr[idx].imshow(act[idx])

In [ ]:
act = activation['conv2'].squeeze()
print(act.shape)
fig, ax = plt.subplots(nrows=4, ncols=4, figsize=(8,8))
for ax, feature in zip(ax.flatten(), act):
    ax.imshow(feature)



### Activation Maximization

In [ ]:
list(model.children())[0]

In [ ]:
class SaveFeatures():
    def __init__(self, module):
        self.hook = module.register_forward_hook(self.hook_fn)
    def hook_fn(self, module, input, output):
        self.features = torch.tensor(output,requires_grad=True).cuda()
    def close(self):
        self.hook.remove()

In [ ]:
activations = SaveFeatures(list(model.children())[0])

In [ ]:
#list(model.children())[0].register_forward_hook()

In [ ]:
#activations.f[0, 1]
#activations.features

In [ ]:
#pip install opencv-python

In [ ]:
import  cv2

In [ ]:
# class FilterVisualizer_Old():
#     def __init__(self,  model,size=28, upscaling_steps=12, upscaling_factor=1.2):
#         self.size, self.upscaling_steps, self.upscaling_factor = size, upscaling_steps, upscaling_factor
#         self.model = model
#         self.model.eval()

#     def visualize(self, layer, filter, lr=0.1, opt_steps=20, blur=None):
#         sz = self.size
#         img = np.uint8(np.random.uniform(150, 180, (1,sz, sz)))/255  # generate random image
#         activations = SaveFeatures(list(self.model.children())[layer])  # register hook
#         print(img.shape)
#         for _ in range(self.upscaling_steps):  # scale the image up upscaling_steps times
# #             train_tfms, val_tfms = tfms_from_model(self.model, sz)
#             img_var = torch.tensor(img[None], dtype=torch.double,requires_grad=True)  # convert image to Variable that requires grad
#             optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
#             for n in range(opt_steps):  # optimize pixel values for opt_steps times
#                 optimizer.zero_grad()
#                 #print(img_var.shape)
#                 self.model(img_var)
#                 loss = -activations.features[0, filter].mean()
#                 loss.backward()
#                 optimizer.step()
#             img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
#             self.output = img
#             sz = int(self.upscaling_factor * sz)  # calculate new image size
#             img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
#             if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
#         self.save(layer, filter)
#         activations.close()
        
#     def save(self, layer, filter):
#         plt.imsave("layer_"+str(layer)+"_filter_"+str(filter)+".jpg", np.clip(self.output, 0, 1))

In [ ]:
img = np.uint8(np.random.uniform(150, 180, (28, 28, 3)))/255
print(img.shape)
print(img[None].shape)

In [ ]:
plt.imshow(img)

In [ ]:
class FilterVisualizer():
    def __init__(self,  model,size=28, upscaling_steps=12, upscaling_factor=1.2):
        self.size, self.upscaling_steps, self.upscaling_factor = size, upscaling_steps, upscaling_factor
        self.model = model
        self.model.eval()

    def visualize(self, layer, filter, lr=0.1, opt_steps=20, blur=None):
        sz = self.size
        img = np.uint8(np.random.uniform(150, 180, (1,sz, sz)))/255  # generate random image
        activations = SaveFeatures(list(self.model.children())[layer])  # register hook
        
        #for _ in range(self.upscaling_steps):  # scale the image up upscaling_steps times
            #train_tfms, val_tfms = tfms_from_model(self.model, sz)
        img_var = torch.tensor(img[None], dtype=torch.double,requires_grad=True)  # convert image to Variable that requires grad
        optimizer = torch.optim.Adam([img_var], lr=lr, weight_decay=1e-6)
        for n in range(opt_steps):  # optimize pixel values for opt_steps times
            optimizer.zero_grad()
            #print(img_var.shape)
            self.model(img_var)
            loss = -activations.features[0, filter].mean()
            loss.backward()
            optimizer.step()
        img = img_var.data.cpu().numpy()[0].transpose(1,2,0)
        print(img.shape)
        self.output = cv2.resize(img, (sz, sz))#img
        sz = int(self.upscaling_factor * sz)  # calculate new image size
        img = cv2.resize(img, (sz, sz), interpolation = cv2.INTER_CUBIC)  # scale image up
        if blur is not None: img = cv2.blur(img,(blur,blur))  # blur image to reduce high frequency patterns
        self.save(layer, filter)
        activations.close()
        
    def save(self, layer, filter):
        #print(self.output.reshape(1,28,28))
        plt.imsave("layer_"+str(layer)+"_filter_"+str(filter)+".jpg",np.clip(self.output, 0, 1))

In [ ]:
model = model.double()
layer = 2
filter = 15
FV = FilterVisualizer(model, upscaling_steps=5, upscaling_factor=1.2)
FV.visualize(layer, filter, blur=None)

In [ ]:
img = cv2.imread("/kaggle/input/gen-imgage/layer_2_filter_0.jpg")

In [ ]:
img = torch.tensor(img)

In [ ]:
img.shape

In [ ]:
21952/(28*28)

#### Saliency Maps

In [ ]:

X=image.reshape(1,1,28,28)
X.requires_grad_()
scores = torch.exp(model(X))
score_max_index = scores.argmax()
score_max = scores[0,score_max_index]



In [ ]:
score_max.backward()


In [ ]:
saliency, _ = torch.max(X.grad.data.abs(),dim=1)


# code to plot the saliency map as a heatmap
plt.imshow(saliency[0], cmap=plt.cm.hot)
# plt.imshow(images[0][1])
plt.axis('off')
plt.show()

In [ ]:
imshowMNIST(images[0][1],labels[0][1])